<a href="https://colab.research.google.com/github/jaime-garvey/web_personalization_portfolio/blob/master/notebooks/optimal_path_MDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import copy

%matplotlib inline

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

import pickle

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Load Data

**Data Structures**

In [0]:
id_id2page_dict = "1DGeJYGuF5fTnruZz84ZaBGelakRcKiKf"
id_possible_actions_dict = "15WUaciYffXqNN9_3t3TGWXJ5FotPWtx0"
#id_transition_matrix = "1Fcph1OO4GUMCeObMleRWX_FnPLSWAuMs"
id_rewards_dict = "1CPKK-kriTdUY1VtWfdCAA3HvZklEa116"

In [0]:
def load_pickle_from_drive(id):
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('Filename.csv')  
  data = pickle.load(open('Filename.csv', 'rb'))
  return data

In [0]:
id2page_dict = load_pickle_from_drive(id_id2page_dict)
possible_actions= load_pickle_from_drive(id_possible_actions_dict)
#transition_matrix = load_pickle_from_drive(id_transition_matrix)
rewards_dict = load_pickle_from_drive(id_rewards_dict)

## Data Preprocessing

In [0]:
terminals = [k for k,v in rewards_dict.items() if v == 1]

In [0]:
possible_actions = possible_actions.to_dict()

# Model

## MDP Class

In [0]:
class MDP:
  '''
  state_0 --> initial state
  terminal_states --> terminal state
  possible_actions --> all actions for every state {state1: {state2, tp}...}
  '''
  
  def __init__(self, state, possible_actions=possible_actions, rewards_dict=rewards_dict, gamma=0.9):
    self.state = state
    self.possible_actions = possible_actions
    self.rewards_dict = rewards_dict
    self.gamma = gamma
    self.terminals = [k for k,v in rewards_dict.items() if v == 1]
     
#       update(self,
#              states_possible_actions=states_possible_actions,
#              terminal_states=terminal_states,
#              states = set(), 
#              reward={})
      
  def reward(self):
    #return numeric reward for state
    return self.rewards_dict[self.state]
  
  def actions(self):
    #takes in state and returns list of possible actions if not a terminal state
    #change if needed
    self
    if self.rewards_dict[self.state] == 1:
      return [None]
    else:
      #self.possible_actions = list(self.possible_actions[self.state].keys())
      self.possible_actions = possible_actions[self.state]
      return self.possible_actions

#   def transition(state, action):
#     #transition probability from state_i to state_j
#     for action in self.possible_actions:
      
#       possible_actions[state][state_j]
    
 

In [0]:
list(policy.keys())

### Helper Functions

In [0]:
##Helper Function to itialize Value Function

def initialize_values(rewards_dict=rewards_dict):
    #function to inital values at 0
    # V(s) only has value if it's not a terminal state
    
    V={}
    
    for state in list(rewards_dict.keys()):
      if rewards_dict[state] == 1:
        V[state] = 0
      else:
        V[state] = np.random.random()
    
    return V

In [0]:
def initialize_policy(possible_actions=possible_actions, terminals=terminals):
  policy = {}
  
  def key_maxval(state):
      
      v=list(possible_actions[state].values())
      k=list(possible_actions[state].keys())
  
      return k[v.index(max(v))]
    
  for state in list(possible_actions.keys()):
    #if state not in terminals:
    policy[state] = key_maxval(state)
    
  return policy

In [0]:
#Initialize Policy (state --> action)
policy = initialize_policy()

In [0]:
#Initialize Value (future rewards)
V = initialize_values()

# Value Iterations

In [0]:
def value_iteration(V=V, gamma=0.9, epsilon=0.001):
  while True:
    
    old_V = V.copy()
    delta = 0
    # V(s) only has value if it's not a terminal state
    for state in list(policy.keys()):
      new_v = float('-inf')
      try:
        #loop and take max value for action 
        for action in list(MDP(state).actions().keys()):
          r = MDP(action).reward()
          v = r + gamma * V[action]
          if v > new_v:
            new_v = v
        V[state] = new_v
        delta = max(delta, np.abs(old_v = V[state]))
        
      except:
        pass
    if delta < epsilon:
      break
         
    #find optimal policy
        
    for state in list(policy.keys()):
      best_action = None
      best_value = float('-inf')
            
            
      # loop through all possible actions to find the best current action
      try:
        for action in list(MDP(state).actions().keys()):
          v = r + gamma * V[action]
          r = MDP(action).reward()

          if v>best_value:
            best_value=v
            best_a = action
        policy[state] = best_a
      except:
        pass


In [0]:
value_iteration()

In [0]:
output = open('policy.pkl', 'wb')
pickle.dump(policy,output)

In [0]:
from google.colab import files
files.download('policy.pkl')

In [0]:
output = open('V.pkl', 'wb')
pickle.dump(V,output)

In [0]:
from google.colab import files
files.download('V.pkl')

In [0]:
def one_step_lookahead(state=None, V=None, discount_factor = 0.9):
    """
    Helper function to  calculate state-value function
    
    Arguments:
        env: openAI GYM Enviorment object
        state: state to consider
        V: Estimated Value for each state. Vector of length nS
        discount_factor: MDP discount factor
        
    Return:
        action_values: Expected value of each action in a state. Vector of length nA
    """
    
    # initialize vector of action values
    V = initialize_values()
    
    # loop over the actions we can take in an enviorment 
    for action,probability in MDP(state).actions().items():
      reward = MDP(action)
      
        # loop over the P_sa distribution.
        for probablity, next_state, reward, info in env.P[state][action]:
             #if we are in state s and take action a. then sum over all the possible states we can land into.
            action_values[action] += probablity * (reward + (discount_factor * V[next_state]))
            
    return action_values

In [0]:
def update_policy(env, policy, V, discount_factor):
    
    """
    Helper function to update a given policy based on given value function.
    
    Arguments:
        env: openAI GYM Enviorment object.
        policy: policy to update.
        V: Estimated Value for each state. Vector of length nS.
        discount_factor: MDP discount factor.
    Return:
        policy: Updated policy based on the given state-Value function 'V'.
    """
    
    for state in range(env.nS):
        # for a given state compute state-action value.
        action_values = one_step_lookahead(env, state, V, discount_factor)
        
        # choose the action which maximizez the state-action value.
        policy[state] =  np.argmax(action_values)
        
    return policy


In [0]:
def value_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    """
    Algorithm to solve MPD.
    
    Arguments:
        env: openAI GYM Enviorment object.
        discount_factor: MDP discount factor.
        max_iteration: Maximum No.  of iterations to run.
        
    Return:
        V: Optimal state-Value function. Vector of lenth nS.
        optimal_policy: Optimal policy. Vector of length nS.
    
    """
    # intialize value fucntion
    V = np.zeros(env.nS)
    
    # iterate over max_iterations
    for i in range(max_iteration):
        
        #  keep track of change with previous value function
        prev_v = np.copy(V) 
    
        # loop over all states
        for state in range(env.nS):
            
            # Asynchronously update the state-action value
            #action_values = one_step_lookahead(env, state, V, discount_factor)
            
            # Synchronously update the state-action value
            action_values = one_step_lookahead(env, state, prev_v, discount_factor)
            
            # select best action to perform based on highest state-action value
            best_action_value = np.max(action_values)
            
            # update the current state-value fucntion
            V[state] =  best_action_value
            
        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            # if values of 'V' not changing after one iteration
            if (np.all(np.isclose(V, prev_v))):
                print('Value converged at iteration %d' %(i+1))
                break

    # intialize optimal policy
    optimal_policy = np.zeros(env.nS, dtype = 'int8')
    
    # update the optimal polciy according to optimal value function 'V'
    optimal_policy = update_policy(env, optimal_policy, V, discount_factor)
    
    return V, optimal_policy

#### CLASS

In [0]:
class Value(MDP):
  '''
  states_actions --> dictionary {state_i: {state_j : transition_probability}}
  id2pages --> dictionary {page id: "Page Title"}
  rewards --> dictionary {state: reward value}
  theta --> threshold for the amount of iterations
  
  
  '''
  V = initialize_values()
  
  def __init__(self, theta=0.00001):

    super().__init__(state_i, possible_actions, terminal_states, gamma)
    
    self.theta = threshold
    self.gamma = gamma
    
  def initialize_values(self):
    #function to inital values at 0
    # V(s) only has value if it's not a terminal state
    
    for page in id2pages.key:
      
      if page in actions():
        
        self.V[page] = np.random.random()
      else:
        self.V[page] = 0      #terminal state
        
      return self.V
  
  
  def state_action(self, state, gamma):
    #Helper function to caluclate Q --> state_value function
    
    
    for action in range(length(self.possible_actions)):
      #loop though all possible actions
      
      for probability, next_state, reward in statespace??
      #if we are in state s and take action a. then sum over all 
      #the possible states we can land into.
      
        state_action[action] += probability * (reward + (discount_factor * V[next_state]))
        
    return q
        
      
  def update_policy(self, policy, V, gamma):
    #Function to update a given policy based on a given value function
    
    for state in range(states.keys):
      action_values = q
      
      policy[state] = np.argmax(action_values)
    
      
  def value_iteration(self, delta, gamma):
    #solve MDP
    
    
    #initialize V
    
    
    #while True or iterate over max_iterations
    while True:
      delta = 0
      
      #iteration counter
      iteration += 1
    
      #keeping track of previous value function
      for state in possible_actions:
        
        old_V = V[state]
        
        # Synchronously update the state-action value
        action_values = q
        
        # select best action to perform based on highest state-action value
        best_action_value = np.max(action_values)
        
        #updtae current state_value function
        
        #V[state] = max(sum([p *(R + gamma * V[state]) for p, s, r in transition states probabilities])
        
        V[state] = best_action_value               

        delta = max(delta, np.abs(V[state] - old_V[state]) #stopping criteria
                    
        if delta < epsilon * (1-gamma)/ gamma:   #epsilon --> small enough
          print("FINAL RESULTS:")
                  print("Iterations: " + str(iteration))
                  print("Delta: " + str(delta))
                  print("Gamma: " + str(gamma))
                  print("Epsilon: " + str(epsilon))
          break
                    
        # intialize optimal policy
     optimal_policy = np.zeros(env.nS, dtype = 'int8')
    
     # update the optimal polciy according to optimal value function 'V'
     optimal_policy = update_policy(env, optimal_policy, V, discount_factor)
    
     return V, optimal_policy
                    
                    
        
        
        
        
        
        
        
        
        V = max(transition_prob * (R + gamma * V(s')))
  
  #Bellman Equation: transition_prob * (R + gamma * V(s'))

In [0]:
# while True:
#         delta = 0
#         u = u1.copy()
#         iteration += 1
#         graph_list.append(u)
#         for s in range(tot_states):
#             reward = r[s]
#             v = np.zeros((1,tot_states))
#             v[0,s] = 1.0
#             u1[s] = return_state_utility(v, T, u, reward, gamma)
#             delta = max(delta, np.abs(u1[s] - u[s])) #Stopping criteria       
#         if delta < epsilon * (1 - gamma) / gamma:



 def return_state_utility(v, T, u, reward, gamma):
    """Return the state utility.

    @param v the state vector
    @param T transition matrix
    @param u utility vector
    @param reward for that state
    @param gamma discount factor
    @return the utility of the state
    """
    action_array = np.zeros(4)
    
    for action in range(0, 4):
        action_array[action] = np.sum(np.multiply(u, np.dot(v, T[:,:,action])))
    return reward + gamma * np.max(action_array)
      